In [ ]:
from dataclasses import dataclass
from pathlib import Path
from typing import Optional

import matplotlib.pylab as plt
import pandas as pd

import get_data
import utils

In [ ]:
get_data.Data(
    quant_file=utils.Configuration.RAW_DATA_PATH.joinpath("results.csv"),
    is_correspondence_file=utils.Configuration.RAW_DATA_PATH.joinpath(
        "is_std_table_correspondence.csv"
    ),
    sample_properties_file=utils.Configuration.RAW_DATA_PATH.joinpath(
        "sample_properties.csv"
    ),
).quant_file